In [12]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.storage import get_availability_model_data, get_demand_model_data
from modules.config import *
from modules.svm import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH,
    SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_availability_model_data,
)

  0%|          | 0/3 [00:00<?, ?it/s]

h3: 8 | t:6 | - linear ✓
h3: 8 | t:6 | - rbf ✓
h3: 8 | t:6 | - poly ✓


In [4]:
results = pd.read_parquet(SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
28,2,29997,463.720999,2.45666,49.799192,0.942428,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-2.821923,-2.886414,-2.811045,-2.827429,0.033092,0,8,6,0.010,NaN
29,2,29997,540.428711,8.05105,71.109758,2.550832,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.001, '...",...,-3.781805,-3.847124,-3.746924,-3.780303,0.039700,0,8,6,0.001,NaN


In [5]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'param_C',
    'param_degree',
    'param_gamma',
]].first()

,mean_test_score,param_C,param_degree,param_gamma
param_kernel,,,,
linear,-9.184871,0.1,NaN,NaN
poly,-5.031879,10.0,5.0,NaN
rbf,-4.852768,10.0,NaN,0.01


In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    42.648204
poly      28.595425
rbf       28.346588
Name: mean_fit_time, dtype: float64

In [9]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [8]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH,
        SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_availability_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [2]:
results = pd.read_parquet(SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_kernel,param_max_iter,...,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,param_gamma
4,0,30000,310.643826,13.076152,48.003316,16.295280,1000.0,3.0,poly,1000000,...,0.000426,340381,8,1,5.296453,2.301402,1.119507,0.561435,0.771392,NaN
2,0,30000,523.972428,2.821114,19.061841,3.267338,1000.0,3.0,poly,1000000,...,0.049703,1000000,7,6,44.422623,6.665030,2.735564,0.347723,0.749638,NaN
0,0,30000,577.687461,3.801555,21.593391,3.182233,1000.0,3.0,poly,1000000,...,0.097518,1000000,7,1,83.411424,9.132985,5.459168,0.584643,0.430922,NaN
1,0,30000,561.610503,7.675842,20.243817,5.246130,1000.0,3.0,poly,1000000,...,0.634361,1000000,7,2,60.470063,7.776250,3.757289,0.464416,0.515026,NaN
3,0,9855,114.029170,0.765888,1.757612,0.414131,1000.0,3.0,poly,1000000,...,0.550676,1000000,7,24,93.021669,9.644774,4.488505,0.336382,0.574531,NaN


In [ ]:
results_first_stage = pd.read_parquet(SVM_FIRST_STAGE_AVAILABILITY_RESULTS_PATH)
results_second_stage = pd.read_parquet(SVM_SECOND_STAGE_AVAILABILITY_RESULTS_PATH)
results_first_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_first_stage_availability_results.parquet"))
results_second_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_second_stage_availability_results.parquet"))

In [15]:
for h3_res, time_interval_length in tqdm(resolutions):
    model_data_train, model_data_test = get_availability_model_data(h3_res, time_interval_length)
    print(f"h3_res: {h3_res}; time_interval_length: {time_interval_length}; data size: {len(model_data_train) + len(model_data_test)}")

  0%|          | 0/9 [00:00<?, ?it/s]

7
1
471798
7
2
235926
7
6
78678
7
24
19710
8
1
2035721
8
2
1017977
8
6
339481
8
24
85045
9
24
263895


In [16]:
for h3_res, time_interval_length in tqdm(resolutions):
    model_data_train, model_data_test = get_demand_model_data(h3_res, time_interval_length)
    print(f"h3_res: {h3_res}; time_interval_length: {time_interval_length}; data size: {len(model_data_train) + len(model_data_test)}")

  0%|          | 0/9 [00:00<?, ?it/s]

h3_res: 7; time_interval_length: 1; data size: 252834
h3_res: 7; time_interval_length: 2; data size: 185117
h3_res: 7; time_interval_length: 6; data size: 105181
h3_res: 7; time_interval_length: 24; data size: 47421
h3_res: 8; time_interval_length: 1; data size: 483664
h3_res: 8; time_interval_length: 2; data size: 449030
h3_res: 8; time_interval_length: 6; data size: 373066
h3_res: 8; time_interval_length: 24; data size: 259991
h3_res: 9; time_interval_length: 24; data size: 474837
